In [1]:
using Turing, Distributions
using PyPlot, PyCall

In [2]:
y = [ 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 3.0, 3.0, 3.0, 2.0, 2.0, 2.0, 1.0, 1.0 ];
N = length(y);
K = 3;

In [3]:
@model BayesHmm(y) = begin
    s = tzeros(Int64, N)
    m = tzeros(Dual, K)
    T = Array{Array}(K)
    for i = 1:K
        T[i] ~ Dirichlet(ones(K)/K)
        # m[i] ~ Normal(1, 0.1) # Defining m this way causes label-switching problem.
        m[i] ~ Normal(i, 0.01)
    end
    s[1] ~ Categorical(ones(Float64, K)/K)
    for i = 2:N
        s[i] ~ Categorical(vec(T[s[i-1]]))
        y[i] ~ Normal(m[s[i]], 0.01)
    end
    return(s, m, T)
end

 Assume - `T` is a parameter
 Assume - `m` is a parameter
 Assume - `s` is a parameter
 Observe - `y` is an observation


BayesHmm (generic function with 4 methods)

In [4]:
g1 = Gibbs(300, HMC(2, 0.2, 5, :m, :T), PG(50, 5, :s))
g2 = PG(50, 1000)

Turing.PG(50,1000,Turing.resampleSystematic,0.5,Set{Any}())

In [ ]:
s1 = @sample(BayesHmm(y), g1);

T[1] = ForwardDiff.Dual{0,Float64}[Dual(0.117584),Dual(0.744224),Dual(0.138192)], T[1] ~ Distributions.Dirichlet{Float64}(alpha=[0.333333,0.333333,0.333333])
m[1] = Dual(0.9967711359430271), m[1] ~ Distributions.Normal{Float64}(μ=1.0, σ=0.01)
T[2] = ForwardDiff.Dual{0,Float64}[Dual(0.0064186),Dual(0.0065692),Dual(0.987012)], T[2] ~ Distributions.Dirichlet{Float64}(alpha=[0.333333,0.333333,0.333333])
m[2] = Dual(2.007863731847725), m[2] ~ Distributions.Normal{Float64}(μ=2.0, σ=0.01)
T[3] = ForwardDiff.Dual{0,Float64}[Dual(0.713092),Dual(0.234245),Dual(0.0526634)], T[3] ~ Distributions.Dirichlet{Float64}(alpha=[0.333333,0.333333,0.333333])
m[3] = Dual(2.9919946593762465), m[3] ~ Distributions.Normal{Float64}(μ=3.0, σ=0.01)
s[1] = Dual(3), s[1] ~ Distributions.Categorical{Float64}(K=3, p=[0.333333,0.333333,0.333333])


In [ ]:
describe(s1)

In [ ]:
m = s1[:m][111];
s = s1[:s][111];
PyPlot.plot(y, linestyle="None", marker="+", color = "r")
PyPlot.plot(m[s], linestyle="-", marker="*", color = "b")

In [ ]:
s2 = @sample(BayesHmm(y), g2);

In [ ]:
describe(s2)

In [ ]:
m = s2[:m][150];
s = s2[:s][150];
PyPlot.plot(y, linestyle="None", marker="+", color = "r")
PyPlot.plot(m[s], linestyle="-", marker="*", color = "b")